In [1]:
!pip install transformers

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip available: 22.3 -> 22.3.1
[notice] To update, run: python3 -m pip install --upgrade pip


In [2]:
import torch
available_gpus = [torch.cuda.device(i) for i in range(torch.cuda.device_count())]
available_gpus

In [3]:
import torch, os, re, pandas as pd, json
import torch
from sklearn.model_selection import train_test_split
from transformers import DataCollatorForLanguageModeling, DataCollatorWithPadding, GPT2Tokenizer, GPT2LMHeadModel, Trainer, TrainingArguments, AutoConfig

import pandas as pd
# from transformers import GPT2LMHeadModel, GPT2Tokenizer
import numpy as np
import random
import torch
from torch.utils.data import Dataset, DataLoader
# from transformers import GPT2Tokenizer, GPT2LMHeadModel, AdamW, get_linear_schedule_with_warmup
from tqdm import tqdm, trange
import torch.nn.functional as F
import csv
from torch import optim
import gc
from torch.nn import DataParallel

/home/ubuntu/.local/lib/python3.8/site-packages/pandas/core/computation/expressions.py:20: UserWarning: Pandas requires version '2.7.3' or newer of 'numexpr' (version '2.7.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


In [4]:
device = "cuda:2" if torch.cuda.is_available() else "cpu"
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# device = torch.device("cuda:0,1,2,3" if torch.cuda.is_available() else "cpu")

In [5]:
# base_tokenizer = GPT2Tokenizer.from_pretrained('gpt2-xl')
base_tokenizer = GPT2Tokenizer.from_pretrained('gpt2-xl')

In [6]:
# head_model = GPT2LMHeadModel.from_pretrained('gpt2-xl').to(device)
head_model = GPT2LMHeadModel.from_pretrained('gpt2-xl')

In [7]:
import os

pos_path = "/home/ubuntu/IMDB_train/pos/"
neg_path = "/home/ubuntu/IMDB_train/neg/"


pos_token_list = []
pos_text_list = []
neg_token_list = []
neg_text_list = []

os.chdir(pos_path)
  

for file in os.listdir():
    if not file.endswith('.txt'):
        continue
    with open(pos_path+file, 'r') as f:
        pos_text = f.read()
        # put into text list
        pos_text_list.append(pos_text)
        #tokenize and put into token list
        #pos_token_list.append(base_tokenizer.encode(pos_text, return_tensors = 'pt'))

os.chdir(neg_path)

for file in os.listdir():
    if not file.endswith('.txt'):
        continue
    with open(neg_path+file, 'r') as f:
        neg_text = f.read()
        # put into text list
        neg_text_list.append(neg_text)
        #tokenize and put into token list
        #neg_token_list.append(base_tokenizer.encode(neg_text_list, return_tensors = 'pt'))

In [8]:
from io import StringIO
from html.parser import HTMLParser

# ref https://stackoverflow.com/questions/753052/strip-html-from-strings-in-python
class MLStripper(HTMLParser):
    def __init__(self):
        super().__init__()
        self.reset()
        self.strict = False
        self.convert_charrefs= True
        self.text = StringIO()
    def handle_data(self, d):
        self.text.write(d)
    def get_data(self):
        return self.text.getvalue()

def strip_tags(html):
    s = MLStripper()
    s.feed(html)
    return s.get_data()
# cleaned = test_text_pos.replace('<br /><br />', ' ')
# print(cleaned)

def clean_imdb(review_list):
    for review in range(len(review_list)):
        cleaned_review = strip_tags(review_list[review])
        review_list[review] = cleaned_review

In [9]:
clean_imdb(pos_text_list)

In [10]:
# df_pos_tokens = pd.DataFrame(pos_token_list, columns = ['tokens'])
# print(df_pos_tokens)
df_pos_text = pd.DataFrame(pos_text_list, columns = ['text'])
print(df_pos_text)

                                                    text
0      Alright this was quite a sensitive little numb...
1      I'm not sure under what circumstances director...
2      Didn't Mystic Pizza win the Oscar for that yea...
3      I really enjoyed this film because I have a tr...
4      Director Douglas Sirk scores again with this, ...
...                                                  ...
12495  When this first came out, my dad brought it ho...
12496  I'm watching the series again now that it's ou...
12497  Note that I did not say that it is better...ju...
12498  I was 10 years old when this show was on TV. B...
12499  I admit I've only seen about three of Shakespe...

[12500 rows x 1 columns]


In [11]:
# text = "I work as a data scientist"
# text_ids = base_tokenizer.encode(text, return_tensors = 'pt')
# print(text_ids.shape)
# print(head_model(text_ids.to(device)).logits.shape)

In [12]:
class ImdbPos(Dataset):  
    def __init__(self, control_code, truncate=False, gpt2_type="gpt2-xl", max_length=1024):

        self.tokenizer = base_tokenizer # can change
        self.text = []

        for row in df_pos_text['text']:
            self.text.append(torch.tensor(
                self.tokenizer.encode((f"<|{control_code}|>{row[:max_length]}<|endoftext|>"))
            ))               
        if truncate:
            self.text = self.text[:20000]
        self.text_count = len(self.text)
        
    def __len__(self):
        return self.text_count

    def __getitem__(self, item):
        return self.text[item]
    
dataset = ImdbPos(df_pos_text['text'], truncate=True, gpt2_type="gpt2-xl")  

In [13]:
def pack_tensor(new_tensor, packed_tensor, max_seq_len):
    # print(packed_tensor)
    # print(new_tensor)
    if packed_tensor is None:
        return new_tensor, True, None
    if new_tensor.size()[1] + packed_tensor.size()[1] > max_seq_len:
        return packed_tensor, False, new_tensor
    else:
        packed_tensor = torch.cat([new_tensor, packed_tensor[:, 1:]], dim=1)
        return packed_tensor, True, None

In [14]:
# t1 = torch.zeros(5)
# t2 = torch.zeros(5)
# t3 = torch.zeros(5)

# first_stacked = torch.stack((t1, t2))
# print(first_stacked.shape)
# second_stacked = torch.cat((first_stacked, t3.unsqueeze(0)))
# print(second_stacked.shape)

In [15]:
def pad_collate(batch):
    max_len = float("-inf")
    stacked_batch = None
    for ten in batch:
        if ten.size()[0] > max_len:
            max_len = ten.size()[0]
    for ten in batch:
        cur_ten_len = ten.size()[0]
        # running_ten = ten
        for cur_ten_idx in range(cur_ten_len, max_len):
            # print(type(ten))
            ten = torch.cat((ten ,torch.tensor(base_tokenizer.encode(base_tokenizer.eos_token))))
        if stacked_batch is None:
            stacked_batch = ten.unsqueeze(0)
        else:
            stacked_batch = torch.cat((stacked_batch, ten.unsqueeze(0)))
            
    return stacked_batch
    # base_tokenizer.eos_token

In [16]:
print(base_tokenizer.encode(base_tokenizer.eos_token)[0])

50256


In [17]:
def train(
    dataset, model, tokenizer,
    batch_size=10, epochs=20, lr=2e-5,
    max_seq_len=1000, warmup_steps=200,
    gpt2_type="gpt2-xl", output_dir=".", output_prefix="wreckgar",
    test_mode=False,save_model_on_epoch=False,
):
    acc_steps = 100
    # device=torch.device("cuda")
    model = DataParallel(model, device_ids = [2])
    model = model.to(device)
    model.train()

    optimizer = optim.RAdam(model.parameters(), lr=lr)
    # scheduler = get_linear_schedule_with_warmup(
    #     optimizer, num_warmup_steps=warmup_steps, num_training_steps=1250*20
    # )

    train_dataloader = DataLoader(dataset, batch_size=batch_size, collate_fn=pad_collate, shuffle=True)

    # train_dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

    loss=0
    accumulating_batch_count = 0
    input_tensor = None

    for epoch in range(epochs):

        print(f"Training epoch {epoch}")
        # print(loss)
        for idx, entry in tqdm(enumerate(train_dataloader)):
            
            #print(len(train_dataloader))
            # print(f"0 {entry}")
            (input_tensor, carry_on, remainder) = pack_tensor(entry, input_tensor, 768)
            # print(f"1 {input_tensor}")
            del entry
            # print(f"2 {input_tensor}")
            del remainder
            # print(f"3 {input_tensor}")
            gc.collect()
            # print("before")
            # print(carry_on)
            # print(idx)
            # print(len(train_dataloader) - 1)
            # print(f"4 {input_tensor}")
            if carry_on and idx != len(train_dataloader) - 1:
                continue
            # print("after")
            # print(f"5 {input_tensor}")
            input_tensor = input_tensor.to(device)
            outputs = model(input_tensor, labels=input_tensor)
            loss = outputs[0]
            if idx%100 == 0:
                print(f"LOSS: {loss}")
            loss.backward()
            del input_tensor
            del outputs
            del loss #?
            gc.collect()
            if (accumulating_batch_count % batch_size) == 0:
                optimizer.step()
                # scheduler.step()
                optimizer.zero_grad(set_to_none=True)
                model.zero_grad(set_to_none=True)

            accumulating_batch_count += 1
            # del input_tensor
            # del outputs
            # del loss #?
            gc.collect()
            input_tensor = None
        loss = 0
        input_tensor = None
        if save_model_on_epoch:
            torch.save(
                model.state_dict(),
                os.path.join(output_dir, f"{output_prefix}-{epoch}.pt"),
            )
    return model

In [18]:
# # pos_imdb_model = head_model
torch.cuda.empty_cache()
t = torch.cuda.get_device_properties(1).total_memory
r = torch.cuda.memory_reserved(1)
a = torch.cuda.memory_allocated(1)
f = r-a  # free inside reserved
print(t/1000000000)
print(r/1000000000)
print(a/1000000000)
print(f/1000000000)

42.358472704
0.0
0.0
0.0


In [19]:
# pos_imdb_model = train(dataset, head_model, base_tokenizer, epochs = 20,  max_seq_len=1000, batch_size=16)

pos_imdb_model = train(dataset, head_model, base_tokenizer, epochs = 5, max_seq_len=1000, batch_size=1)

Training epoch 0


100it [01:02,  1.78it/s]

LOSS: 1.876409888267517


400it [04:03,  1.96it/s]

LOSS: 1.5835665464401245


...

LOSS: 0.8172794580459595


1400it [14:23,  1.77it/s]

LOSS: 1.3546288013458252


1440it [14:50,  1.62it/s]


KeyboardInterrupt: 

In [20]:
torch.save(head_model.state_dict(), "/home/ubuntu/pos_imdb_model_intital_fine_tune_2")